# 리트리버 성능평가

In [148]:
import pandas as pd
import os
from copy import deepcopy

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

In [149]:
import ast
import re

def preprocess(text):
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\\n", " ", text)
    text = re.sub(r"#", " ", text)
    text = re.sub(
        r"[^A-Za-z0-9가-힣.?!,()~‘’“”" ":%&《》〈〉''㈜·\-'+\s一-龥サマーン]",
        "",
        text,
    )  # サマーン 는 predictions.json에 있었음
    text = re.sub(r"\s+", " ", text).strip()  # 두 개 이상의 연속된 공백을 하나로 치환
    # text = re.sub(r"[^A-Za-z0-9가-힣.?!,()~‘’“”"":%&《》〈〉''㈜·\-\'+\s一-龥]", "", text)

    return text


def evaluate_retriever(df, topk=100, is_preprocess=False, is_title=False):
    df["context"] = df["context"].apply(
        lambda x: ast.literal_eval(x)
    )
    correct_ids = []
    wrong_ids = []
    for idx,row in df.iterrows():
        org_context = (
            preprocess(row["original_context"])
            if is_preprocess
            else row["original_context"]
        )
        if is_title:
            for passage in row["context"][:topk]:
                if passage.split(": ")[1] == org_context:
                    correct_ids.append(row["id"])
                    break
            else:
                wrong_ids.append(row["id"])
        else:
            if org_context in row["context"][:topk]:
                correct_ids.append(row["id"])
            else:
                wrong_ids.append(row["id"])

    print(
        f"전체: {len(df)}, 맞은 개수: {len(correct_ids)}, 틀린 개수: {len(wrong_ids)}, 정답률: {(len(correct_ids)/len(df)):.4%}"
    )
    return correct_ids, wrong_ids

## Elasticsearch

In [140]:
es_path = os.path.join(parent_dir, "data", "es_train")
df_es_100 = pd.read_csv(os.path.join(es_path, "train_es_100.csv"))
dfs = []
for i in range(10):
    dfs.append(deepcopy(df_es_100))

In [142]:
for df, topk in zip(deepcopy(dfs), [k for k in range(10, 110, 10)]):
    _, _ = evaluate_retriever(df, topk=topk, is_preprocess=True)

전체: 240, 맞은 개수: 222, 틀린 개수: 18, 정답률: 92.5000%
전체: 240, 맞은 개수: 227, 틀린 개수: 13, 정답률: 94.5833%
전체: 240, 맞은 개수: 231, 틀린 개수: 9, 정답률: 96.2500%
전체: 240, 맞은 개수: 232, 틀린 개수: 8, 정답률: 96.6667%
전체: 240, 맞은 개수: 232, 틀린 개수: 8, 정답률: 96.6667%
전체: 240, 맞은 개수: 234, 틀린 개수: 6, 정답률: 97.5000%
전체: 240, 맞은 개수: 234, 틀린 개수: 6, 정답률: 97.5000%
전체: 240, 맞은 개수: 234, 틀린 개수: 6, 정답률: 97.5000%
전체: 240, 맞은 개수: 235, 틀린 개수: 5, 정답률: 97.9167%
전체: 240, 맞은 개수: 235, 틀린 개수: 5, 정답률: 97.9167%


## BM25

In [145]:
bm25_path = os.path.join(parent_dir, "data", "bm25_train")

df_bm25_100 = pd.read_csv(os.path.join(bm25_path, "train_bm25_100.csv"))
dfs = []
for i in range(10):
    dfs.append(deepcopy(df_bm25_100))

In [147]:
for df, topk in zip(deepcopy(dfs), [k for k in range(10, 110, 10)]):
    _, _ = evaluate_retriever(df, topk=topk, is_title=True)

전체: 240, 맞은 개수: 203, 틀린 개수: 37, 정답률: 84.5833%
전체: 240, 맞은 개수: 210, 틀린 개수: 30, 정답률: 87.5000%
전체: 240, 맞은 개수: 215, 틀린 개수: 25, 정답률: 89.5833%
전체: 240, 맞은 개수: 216, 틀린 개수: 24, 정답률: 90.0000%
전체: 240, 맞은 개수: 217, 틀린 개수: 23, 정답률: 90.4167%
전체: 240, 맞은 개수: 217, 틀린 개수: 23, 정답률: 90.4167%
전체: 240, 맞은 개수: 217, 틀린 개수: 23, 정답률: 90.4167%
전체: 240, 맞은 개수: 217, 틀린 개수: 23, 정답률: 90.4167%
전체: 240, 맞은 개수: 218, 틀린 개수: 22, 정답률: 90.8333%
전체: 240, 맞은 개수: 218, 틀린 개수: 22, 정답률: 90.8333%


## Rerank-Elasticsearch

In [127]:
es_path = os.path.join(parent_dir, "data", "es_train")

df_es_20 = pd.read_csv(os.path.join(es_path, "not_train_es_20_rerank_5.csv"))
df_es_30 = pd.read_csv(os.path.join(es_path, "not_train_es_30_rerank_5.csv"))
df_es_40 = pd.read_csv(os.path.join(es_path, "not_train_es_40_rerank_5.csv"))
df_es_50 = pd.read_csv(os.path.join(es_path, "not_train_es_50_rerank_5.csv"))
df_es_60 = pd.read_csv(os.path.join(es_path, "not_train_es_60_rerank_5.csv"))
df_es_70 = pd.read_csv(os.path.join(es_path, "not_train_es_70_rerank_5.csv"))
df_es_80 = pd.read_csv(os.path.join(es_path, "not_train_es_80_rerank_5.csv"))
df_es_90 = pd.read_csv(os.path.join(es_path, "not_train_es_90_rerank_5.csv"))
df_es_100 = pd.read_csv(os.path.join(es_path, "not_train_es_100_rerank_5.csv"))

dfs = [
    df_es_20,
    df_es_30,
    df_es_40,
    df_es_50,
    df_es_60,
    df_es_70,
    df_es_80,
    df_es_90,
    df_es_100,
]

In [128]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=5, is_preprocess=True)

전체: 240, 맞은 개수: 220, 틀린 개수: 20, 정답률: 91.6667%
전체: 240, 맞은 개수: 223, 틀린 개수: 17, 정답률: 92.9167%
전체: 240, 맞은 개수: 222, 틀린 개수: 18, 정답률: 92.5000%
전체: 240, 맞은 개수: 222, 틀린 개수: 18, 정답률: 92.5000%
전체: 240, 맞은 개수: 224, 틀린 개수: 16, 정답률: 93.3333%
전체: 240, 맞은 개수: 222, 틀린 개수: 18, 정답률: 92.5000%
전체: 240, 맞은 개수: 222, 틀린 개수: 18, 정답률: 92.5000%
전체: 240, 맞은 개수: 222, 틀린 개수: 18, 정답률: 92.5000%
전체: 240, 맞은 개수: 222, 틀린 개수: 18, 정답률: 92.5000%


In [129]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=4, is_preprocess=True)

전체: 240, 맞은 개수: 218, 틀린 개수: 22, 정답률: 90.8333%
전체: 240, 맞은 개수: 220, 틀린 개수: 20, 정답률: 91.6667%
전체: 240, 맞은 개수: 221, 틀린 개수: 19, 정답률: 92.0833%
전체: 240, 맞은 개수: 219, 틀린 개수: 21, 정답률: 91.2500%
전체: 240, 맞은 개수: 221, 틀린 개수: 19, 정답률: 92.0833%
전체: 240, 맞은 개수: 220, 틀린 개수: 20, 정답률: 91.6667%
전체: 240, 맞은 개수: 220, 틀린 개수: 20, 정답률: 91.6667%
전체: 240, 맞은 개수: 220, 틀린 개수: 20, 정답률: 91.6667%
전체: 240, 맞은 개수: 220, 틀린 개수: 20, 정답률: 91.6667%


In [130]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=3, is_preprocess=True)

전체: 240, 맞은 개수: 215, 틀린 개수: 25, 정답률: 89.5833%
전체: 240, 맞은 개수: 215, 틀린 개수: 25, 정답률: 89.5833%
전체: 240, 맞은 개수: 214, 틀린 개수: 26, 정답률: 89.1667%
전체: 240, 맞은 개수: 214, 틀린 개수: 26, 정답률: 89.1667%
전체: 240, 맞은 개수: 216, 틀린 개수: 24, 정답률: 90.0000%
전체: 240, 맞은 개수: 215, 틀린 개수: 25, 정답률: 89.5833%
전체: 240, 맞은 개수: 214, 틀린 개수: 26, 정답률: 89.1667%
전체: 240, 맞은 개수: 215, 틀린 개수: 25, 정답률: 89.5833%
전체: 240, 맞은 개수: 215, 틀린 개수: 25, 정답률: 89.5833%


In [131]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=2, is_preprocess=True)

전체: 240, 맞은 개수: 206, 틀린 개수: 34, 정답률: 85.8333%
전체: 240, 맞은 개수: 209, 틀린 개수: 31, 정답률: 87.0833%
전체: 240, 맞은 개수: 209, 틀린 개수: 31, 정답률: 87.0833%
전체: 240, 맞은 개수: 208, 틀린 개수: 32, 정답률: 86.6667%
전체: 240, 맞은 개수: 210, 틀린 개수: 30, 정답률: 87.5000%
전체: 240, 맞은 개수: 208, 틀린 개수: 32, 정답률: 86.6667%
전체: 240, 맞은 개수: 208, 틀린 개수: 32, 정답률: 86.6667%
전체: 240, 맞은 개수: 207, 틀린 개수: 33, 정답률: 86.2500%
전체: 240, 맞은 개수: 207, 틀린 개수: 33, 정답률: 86.2500%


In [132]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=1, is_preprocess=True)

전체: 240, 맞은 개수: 190, 틀린 개수: 50, 정답률: 79.1667%
전체: 240, 맞은 개수: 191, 틀린 개수: 49, 정답률: 79.5833%
전체: 240, 맞은 개수: 191, 틀린 개수: 49, 정답률: 79.5833%
전체: 240, 맞은 개수: 191, 틀린 개수: 49, 정답률: 79.5833%
전체: 240, 맞은 개수: 191, 틀린 개수: 49, 정답률: 79.5833%
전체: 240, 맞은 개수: 191, 틀린 개수: 49, 정답률: 79.5833%
전체: 240, 맞은 개수: 191, 틀린 개수: 49, 정답률: 79.5833%
전체: 240, 맞은 개수: 191, 틀린 개수: 49, 정답률: 79.5833%
전체: 240, 맞은 개수: 191, 틀린 개수: 49, 정답률: 79.5833%


## Rerank-BM25

In [111]:
bm25_path = os.path.join(parent_dir, "data", "bm25_train")

df_bm25_20 = pd.read_csv(os.path.join(bm25_path, "not_train_bm25_20_rerank_5.csv"))
df_bm25_30 = pd.read_csv(os.path.join(bm25_path, "not_train_bm25_30_rerank_5.csv"))
df_bm25_40 = pd.read_csv(os.path.join(bm25_path, "not_train_bm25_40_rerank_5.csv"))
df_bm25_50 = pd.read_csv(os.path.join(bm25_path, "not_train_bm25_50_rerank_5.csv"))
df_bm25_60 = pd.read_csv(os.path.join(bm25_path, "not_train_bm25_60_rerank_5.csv"))
df_bm25_70 = pd.read_csv(os.path.join(bm25_path, "not_train_bm25_70_rerank_5.csv"))
df_bm25_80 = pd.read_csv(os.path.join(bm25_path, "not_train_bm25_80_rerank_5.csv"))
df_bm25_90 = pd.read_csv(os.path.join(bm25_path, "not_train_bm25_90_rerank_5.csv"))
df_bm25_100 = pd.read_csv(os.path.join(bm25_path, "not_train_bm25_100_rerank_5.csv"))

dfs = [
    df_bm25_20,
    df_bm25_30,
    df_bm25_40,
    df_bm25_50,
    df_bm25_60,
    df_bm25_70,
    df_bm25_80,
    df_bm25_90,
    df_bm25_100,
]

In [113]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, is_title=True)

전체: 240, 맞은 개수: 205, 틀린 개수: 35, 정답률: 85.4167%
전체: 240, 맞은 개수: 207, 틀린 개수: 33, 정답률: 86.2500%
전체: 240, 맞은 개수: 207, 틀린 개수: 33, 정답률: 86.2500%
전체: 240, 맞은 개수: 208, 틀린 개수: 32, 정답률: 86.6667%
전체: 240, 맞은 개수: 208, 틀린 개수: 32, 정답률: 86.6667%
전체: 240, 맞은 개수: 208, 틀린 개수: 32, 정답률: 86.6667%
전체: 240, 맞은 개수: 208, 틀린 개수: 32, 정답률: 86.6667%
전체: 240, 맞은 개수: 209, 틀린 개수: 31, 정답률: 87.0833%
전체: 240, 맞은 개수: 209, 틀린 개수: 31, 정답률: 87.0833%


In [114]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=4, is_title=True)

전체: 240, 맞은 개수: 204, 틀린 개수: 36, 정답률: 85.0000%
전체: 240, 맞은 개수: 206, 틀린 개수: 34, 정답률: 85.8333%
전체: 240, 맞은 개수: 205, 틀린 개수: 35, 정답률: 85.4167%
전체: 240, 맞은 개수: 206, 틀린 개수: 34, 정답률: 85.8333%
전체: 240, 맞은 개수: 206, 틀린 개수: 34, 정답률: 85.8333%
전체: 240, 맞은 개수: 205, 틀린 개수: 35, 정답률: 85.4167%
전체: 240, 맞은 개수: 204, 틀린 개수: 36, 정답률: 85.0000%
전체: 240, 맞은 개수: 205, 틀린 개수: 35, 정답률: 85.4167%
전체: 240, 맞은 개수: 205, 틀린 개수: 35, 정답률: 85.4167%


In [115]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=3, is_title=True)

전체: 240, 맞은 개수: 203, 틀린 개수: 37, 정답률: 84.5833%
전체: 240, 맞은 개수: 206, 틀린 개수: 34, 정답률: 85.8333%
전체: 240, 맞은 개수: 203, 틀린 개수: 37, 정답률: 84.5833%
전체: 240, 맞은 개수: 204, 틀린 개수: 36, 정답률: 85.0000%
전체: 240, 맞은 개수: 204, 틀린 개수: 36, 정답률: 85.0000%
전체: 240, 맞은 개수: 204, 틀린 개수: 36, 정답률: 85.0000%
전체: 240, 맞은 개수: 203, 틀린 개수: 37, 정답률: 84.5833%
전체: 240, 맞은 개수: 202, 틀린 개수: 38, 정답률: 84.1667%
전체: 240, 맞은 개수: 202, 틀린 개수: 38, 정답률: 84.1667%


In [116]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=2, is_title=True)

전체: 240, 맞은 개수: 195, 틀린 개수: 45, 정답률: 81.2500%
전체: 240, 맞은 개수: 197, 틀린 개수: 43, 정답률: 82.0833%
전체: 240, 맞은 개수: 197, 틀린 개수: 43, 정답률: 82.0833%
전체: 240, 맞은 개수: 198, 틀린 개수: 42, 정답률: 82.5000%
전체: 240, 맞은 개수: 197, 틀린 개수: 43, 정답률: 82.0833%
전체: 240, 맞은 개수: 197, 틀린 개수: 43, 정답률: 82.0833%
전체: 240, 맞은 개수: 196, 틀린 개수: 44, 정답률: 81.6667%
전체: 240, 맞은 개수: 197, 틀린 개수: 43, 정답률: 82.0833%
전체: 240, 맞은 개수: 197, 틀린 개수: 43, 정답률: 82.0833%


In [117]:
for df in deepcopy(dfs):
    _, _ = evaluate_retriever(df, topk=1, is_title=True)

전체: 240, 맞은 개수: 171, 틀린 개수: 69, 정답률: 71.2500%
전체: 240, 맞은 개수: 174, 틀린 개수: 66, 정답률: 72.5000%
전체: 240, 맞은 개수: 174, 틀린 개수: 66, 정답률: 72.5000%
전체: 240, 맞은 개수: 175, 틀린 개수: 65, 정답률: 72.9167%
전체: 240, 맞은 개수: 174, 틀린 개수: 66, 정답률: 72.5000%
전체: 240, 맞은 개수: 174, 틀린 개수: 66, 정답률: 72.5000%
전체: 240, 맞은 개수: 174, 틀린 개수: 66, 정답률: 72.5000%
전체: 240, 맞은 개수: 174, 틀린 개수: 66, 정답률: 72.5000%
전체: 240, 맞은 개수: 173, 틀린 개수: 67, 정답률: 72.0833%
